## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-28-20-52-00 +0000,wsj,"Stock Market Today: S&P, Nasdaq Edge Higher to...",https://www.wsj.com/livecoverage/stock-market-...
1,2025-07-28-20-51-36 +0000,nypost,Vance claims ‘Trump has nothing to hide’ in Ep...,https://nypost.com/2025/07/28/us-news/jd-vance...
2,2025-07-28-20-50-30 +0000,nyt,Veteran C.I.A. Official to Retire After Losing...,https://www.nytimes.com/2025/07/28/us/politics...
3,2025-07-28-20-47-00 +0000,wsj,"Verizon Hikes In-Office Requirement, Leases La...",https://www.wsj.com/real-estate/commercial/ver...
4,2025-07-28-20-45-31 +0000,bbc,Coach Deion Sanders 'cured from cancer' after ...,https://www.bbc.com/news/articles/cx29lgxz4y4o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,49
10,deal,25
9,trade,24
103,gaza,18
6,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
136,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...,127
63,2025-07-28-18-24-16 +0000,nypost,France presses EU to unload ‘trade bazooka’ on...,https://nypost.com/2025/07/28/business/france-...,124
247,2025-07-27-22-21-28 +0000,wapo,"Trump, E.U. reach contours of trade deal with ...",https://www.washingtonpost.com/politics/2025/0...,114
167,2025-07-28-11-56-36 +0000,nyt,"Europe, Worried by Other Global Crises, Agrees...",https://www.nytimes.com/2025/07/28/world/europ...,113
91,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...,110


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
136,127,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...
91,61,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...
157,50,2025-07-28-13-14-04 +0000,nypost,Starving Palestinians swarm aid trucks in Gaza...,https://nypost.com/2025/07/28/world-news/pales...
0,47,2025-07-28-20-52-00 +0000,wsj,"Stock Market Today: S&P, Nasdaq Edge Higher to...",https://www.wsj.com/livecoverage/stock-market-...
96,42,2025-07-28-16-50-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...
133,40,2025-07-28-15-04-00 +0000,wsj,Former Gov. Roy Cooper of North Carolina launc...,https://www.wsj.com/politics/elections/roy-coo...
79,36,2025-07-28-18-02-44 +0000,nyt,Thailand and Cambodia Agree to Halt Fighting T...,https://www.nytimes.com/2025/07/28/world/asia/...
28,35,2025-07-28-19-54-48 +0000,nypost,Pervert Delta Airlines pilot hit with child se...,https://nypost.com/2025/07/28/us-news/delta-ai...
84,31,2025-07-28-17-44-19 +0000,nypost,Hamas-cheering Irish band Kneecap to play pair...,https://nypost.com/2025/07/28/us-news/hamas-ch...
249,27,2025-07-27-21-52-19 +0000,latimes,Hotshot firefighter critically injured in Nort...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
